__Read this__

* http://csbio.unc.edu/mcmillan/Comp555S16/Lecture14.html
* https://www.cs.cmu.edu/~ckingsf/bioinfo-lectures/gaps.pdf
* http://www.bioinfo.rpi.edu/bystrc/courses/biol4540/lecture4.pdf

In [1]:
import pandas as pd
import numpy as np
def createScorMat(match = 1,mismatch = 0,nts = ['A','C','T','G']):
    """
    Create scoring matrix for nucleotides with a given match and mismatch values
    """
    scoringMatrix = np.zeros((len(nts),len(nts)),)
    np.fill_diagonal(scoringMatrix,match)
    scoringMatrix
    scoringMatrix[scoringMatrix == 0] = mismatch
    scoringMatrix = pd.DataFrame(scoringMatrix,index=nts,columns=nts)
    return scoringMatrix

#### Dynamic programming implementation
def scoringMatrix_local(x,y,subMat,indel):
    m = len(x) + 1
    n = len(y) + 1
    globMax = (0,0)
    scoringMat=np.zeros((n,m))
    for i in range(1,m):
        #print(x[i-1])
        for j in range(1,n):
            if x[i-1] == y[j-1]:
                #scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                scoringMat[j][i] = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            else:
                mismatch = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                left = scoringMat[j-1][i] + indel
                up = scoringMat[j][i-1] + indel
                scoringMat[j][i] = max(mismatch,left,up,0)
            if scoringMat[j][i] >= scoringMat[globMax[1]][globMax[0]]:
                globMax = (i,j)
    return scoringMat,globMax


def scoringMatrix_global(x,y,subMat,indel):
    m = len(x) + 1
    n = len(y) + 1
    scoringMat=np.zeros((n,m))
    scoringMat[:][0] = [0] + list(np.cumsum([indel for _ in x]))
    scoringMat[:,0] = [0] + list(np.cumsum([indel for _ in y]))
    for i in range(1,m):
        #print(x[i-1])
        for j in range(1,n):
            if x[i-1] == y[j-1]:
                #scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                scoringMat[j][i] = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            else:
                mismatch = scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
                left = scoringMat[j-1][i] + indel
                up = scoringMat[j][i-1] + indel
                scoringMat[j][i] = max(mismatch,left,up)
    return scoringMat


def backtrack_local(x,y,sm,maxLocs):
    i = maxLocs[0]
    j = maxLocs[1]
    w=''
    z=''
    z,w
    while i*j > 0:
        if x[i-1] == y[j-1]:
            w += x[i-1]
            z += y[j-1]
            i -= 1
            j -= 1
        else:
            left = sm[j-1][i]
            up = sm[j][i-1]
            diag = sm[j-1][i-1]
            whichmax = np.argmax([left,up,diag])
            if whichmax == 0:
                w += '-'
                z += y[j-1]
                j -= 1
            elif whichmax == 1:
                z += '-'
                w += x[i-1]
                i -= 1
            else:
                w += x[i-1]
                z += y[j-1]
                i -= 1
                j -= 1
        if sm[j][i] == 0:
            break
    return w[::-1],z[::-1]

def backtrack_global(x,y,sm,subMat,indel):
    i = len(x)
    j = len(y)
    w=''
    z=''
    z,w
    while i*j > 0:
        if x[i-1] == y[j-1]:
            w += x[i-1]
            z += y[j-1]
            i -= 1
            j -= 1
        else:
            left = sm[j-1][i] + indel
            up = sm[j][i-1] + indel
            diag = sm[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            whichmax = np.argmax([left,up,diag])
            if whichmax == 0:
                w += '-'
                z += y[j-1]
                j -= 1
            elif whichmax == 1:
                z += '-'
                w += x[i-1]
                i -= 1
            else:
                w += x[i-1]
                z += y[j-1]
                i -= 1
                j -= 1
    if j==0 and i>0:
        w = x[:i]+w[::-1]
        z = '-' * i + z[::-1]
    elif i==0 and j>0:
        z = y[:i]+z[::-1]
        w = '-' * j + w[::-1]
    else:
        w = w[::-1]
        z = z[::-1]
    return w,z

def locAl(x,y,subMat,indel):
    '''
    A wrapper for DP implementation of Smith-Waterman
    '''
    sm, maxLocs = scoringMatrix_local(x,y,subMat,indel)
    z,w = backtrack_local(x,y,sm,maxLocs)
    return z,w,sm[maxLocs[1],maxLocs[0]],sm

def globAl(x,y,subMat,indel):
    '''
    A wrapper for DP implementation of Smith-Waterman
    '''
    sm = scoringMatrix_global(x,y,subMat,indel)
    z,w = backtrack_global(x,y,sm,subMat,indel)
    return z,w,sm[len(y),len(x)]

def vizScoringMat(x,y,scoringMat):
    return pd.DataFrame(scoringMat,index=['']+[_ for _ in y],
             columns=['']+[_ for _ in x])

def readBlosum62():
    '''
    Read scoring matrix
    '''
    with open('blosum62.txt') as matrix_file:
        matrix = matrix_file.read()
        lines = matrix.strip().split('\n')

    blosum={}
    cols = lines[0].split()

    for row in lines[1:]:       
        idx = row[0]
        vals = row[1:].split()
        vals = [int(_) for _ in vals]
        #print(idx,vals)
        blosum[idx]=vals

    blosum = pd.DataFrame.from_dict(blosum,orient='index',columns=cols)
    return blosum

In [2]:
blosum = readBlosum62()

In [3]:
### Example from https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5
## Problem 3: Overlap Alignment
x='PRTEINS'
y='PRTWPSEIN'
indel = -8
subMat = blosum
locAl(x,y,subMat,indel)

('PRT',
 'PRT',
 17.0,
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., 12.,  4.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  4., 17.,  9.,  1.,  0.,  1.],
        [ 0.,  0.,  0.,  9., 14.,  6.,  0.,  0.],
        [ 0.,  7.,  0.,  1.,  8., 11.,  4.,  0.],
        [ 0.,  0.,  6.,  1.,  1.,  6., 12.,  8.],
        [ 0.,  0.,  0.,  5.,  6.,  0.,  6., 12.],
        [ 0.,  0.,  0.,  0.,  2., 10.,  2.,  4.],
        [ 0.,  0.,  0.,  0.,  0.,  2., 16.,  8.]]))

In [25]:
### Example from https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5
## Problem 3: Overlap Alignment
x='ATTGACCTGA'
y='ATCCTGA'
indel = -1
gap = -1
subMat = createScorMat(match = 1,mismatch = -2,nts = list(set(y) | set(x)))
globAl(x,y,subMat,indel),

(('ATTGACCTGA', 'A-T--CCTGA', 4.0),)

In [4]:
### Example from https://www.bioinformaticsalgorithms.org/bioinformatics-chapter-5
## Problem 3: Overlap Alignment
x='PRTEINS'
y='PRTWPSEIN'
indel = -11
gap = -1
subMat = blosum
sm,_ = scoringMatrix_local(x,y,subMat,indel)
backtrack_local(x,y,sm,(len(x),len(y)))

('PRTEINS', 'PS-EIN-')

In [24]:
#http://csbio.unc.edu/mcmillan/Comp555S16/Lecture14.html
#http://rosalind.info/problems/ba5j/

(7, 9)

In [6]:
##http://csbio.unc.edu/mcmillan/Comp555S16/Lecture14.html

m = len(x) + 1
n = len(y) + 1

uTable=np.zeros((n,m)) # Keeps track of w gaps 
sTable=np.zeros((n,m)) # Main scoring matrix
tTable=np.zeros((n,m)) # Keeps track of v gaps

### Initialize 
# main
sTable[:][0] = [0] + list(np.cumsum([0 for _ in x]))
sTable[:,0] = [0] + list(np.cumsum([0 for _ in y]))
# left
tTable[:][0] = [0] + list(np.cumsum([0 for _ in x]))
tTable[:,0] = [0] + list(np.cumsum([0 for _ in y]))
# up
uTable[:][0] = [0] + list(np.cumsum([0 for _ in x]))
uTable[:,0] = [0] + list(np.cumsum([0 for _ in y]))
#########
for i in range(1,m):
    #print(x[i-1])
    for j in range(1,n):
        if x[i-1] == y[j-1]:
            #scoringMat[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            sTable[j][i] = sTable[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
        else:
            ### up
            contGapV = tTable[j-1][i-1] + gap
            startGapV = sTable[j-1][i-1] + indel + gap
            tTable[j][i] = max(contGapV, startGapV)

            ### left
            contGapW = uTable[j-1][i-1] + gap
            startGapW = sTable[j-1][i-1] + indel + gap
            uTable[j][i] = max(contGapW, startGapW)

            ###
            mismatch = sTable[j-1][i-1] + subMat.loc[x[i-1]][y[j-1]]
            left = tTable[j][i]
            up = uTable[j][i]
            sTable[j][i] = max(mismatch,left,up)
sTable, tTable, uTable

(array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  7., -1., -1., -1., -1., -1., -1.],
        [ 0., -1., 12., -2., -1., -2., -1., -2.],
        [ 0., -1., -2., 17., -3., -2., -2.,  0.],
        [ 0., -1., -2., -3., 14., -4., -4., -4.],
        [ 0.,  7., -2., -3., -4., 11., -5., -5.],
        [ 0., -1.,  6., -1., -3., -5., 12., -1.],
        [ 0., -1., -1.,  5.,  4., -5., -5., 12.],
        [ 0., -1., -2., -2.,  2.,  8., -6., -7.],
        [ 0., -1., -1., -2., -2., -1., 14., -5.]]),
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1., -1., -1., -1., -1., -1.],
        [ 0., -1.,  0., -2., -2., -2., -2., -2.],
        [ 0., -1., -2.,  0., -3., -3., -3., -3.],
        [ 0., -1., -2., -3.,  5., -4., -4., -4.],
        [ 0.,  0., -2., -3., -4.,  4., -5., -5.],
        [ 0., -1., -1., -3., -4., -5.,  3.,  0.],
        [ 0., -1., -2., -2.,  0., -5., -6.,  2.],
        [ 0., -1., -2., -3., -3.,  0., -6., -7.],
        [ 0., -1., -2., -3., -4., -4.,  0., -7.]